# Portfolio Asset Clustering Project (Canary Group)

## Dataset Preparation and Feature Engineering

In [24]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Loading data from Excel...")
# Read the Excel file created by R
df = pd.read_excel('sp500_stock_data.xlsx', sheet_name='Stock_Data')

print(f"Loaded {len(df):,} rows for {df['Ticker'].nunique()} stocks")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")

# Convert Date to datetime if not already
df['Date'] = pd.to_datetime(df['Date'])

# Sort by ticker and date
df = df.sort_values(['Ticker', 'Date'])

Loading data from Excel...
Loaded 249,024 rows for 500 stocks
Date range: 2023-12-04 00:00:00 to 2025-11-28 00:00:00


### Feature calculation functions

In [25]:
# Feature calculation functions
def calculate_returns_features(group):
    """Calculate return-based features"""
    features = {}
    
    returns = group['Daily_Return'].dropna()
    
    if len(returns) < 30:
        return pd.Series(features)
    
    # Basic return metrics
    features['daily_return_mean'] = returns.mean()
    features['daily_return_std'] = returns.std()
    features['annualized_return'] = returns.mean() * 252
    features['annualized_volatility'] = returns.std() * np.sqrt(252)
    
    # Period returns
    if len(group) >= 21:
        features['return_1m'] = (group['Close'].iloc[-1] / group['Close'].iloc[-21]) - 1
    if len(group) >= 63:
        features['return_3m'] = (group['Close'].iloc[-1] / group['Close'].iloc[-63]) - 1
    if len(group) >= 126:
        features['return_6m'] = (group['Close'].iloc[-1] / group['Close'].iloc[-126]) - 1
    if len(group) >= 252:
        features['return_1y'] = (group['Close'].iloc[-1] / group['Close'].iloc[-252]) - 1
    
    # Cumulative return
    features['cumulative_return'] = (group['Close'].iloc[-1] / group['Close'].iloc[0]) - 1
    
    return pd.Series(features)

def calculate_risk_features(group):
    """Calculate risk metrics"""
    features = {}
    
    returns = group['Daily_Return'].dropna()
    
    if len(returns) < 30:
        return pd.Series(features)
    
    # Downside metrics
    negative_returns = returns[returns < 0]
    features['downside_deviation'] = negative_returns.std() * np.sqrt(252)
    features['semi_variance'] = negative_returns.var()
    
    # Maximum drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    features['max_drawdown'] = drawdown.min()
    
    # VaR and CVaR
    features['var_95'] = returns.quantile(0.05)
    features['var_99'] = returns.quantile(0.01)
    features['cvar_95'] = returns[returns <= features['var_95']].mean()
    features['cvar_99'] = returns[returns <= features['var_99']].mean()
    
    return pd.Series(features)

def calculate_risk_adjusted_features(group, risk_free_rate=0.04):
    """Calculate risk-adjusted metrics"""
    features = {}
    
    returns = group['Daily_Return'].dropna()
    
    if len(returns) < 30:
        return pd.Series(features)
    
    ann_return = returns.mean() * 252
    ann_vol = returns.std() * np.sqrt(252)
    
    # Sharpe ratio
    if ann_vol > 0:
        features['sharpe_ratio'] = (ann_return - risk_free_rate) / ann_vol
    
    # Sortino ratio
    negative_returns = returns[returns < 0]
    downside_std = negative_returns.std() * np.sqrt(252)
    if downside_std > 0:
        features['sortino_ratio'] = (ann_return - risk_free_rate) / downside_std
    
    # Calmar ratio
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_dd = drawdown.min()
    if max_dd < 0:
        features['calmar_ratio'] = ann_return / abs(max_dd)
    
    # Omega ratio
    threshold = 0
    excess_returns = returns - threshold
    gains = excess_returns[excess_returns > 0].sum()
    losses = -excess_returns[excess_returns < 0].sum()
    if losses > 0:
        features['omega_ratio'] = gains / losses
    
    return pd.Series(features)

def calculate_technical_features(group):
    """Calculate technical indicators"""
    features = {}
    
    close = group['Close']
    
    if len(close) < 50:
        return pd.Series(features)
    
    # Moving averages
    features['sma_20'] = close.rolling(window=20).mean().iloc[-1] / close.iloc[-1]
    features['sma_50'] = close.rolling(window=50).mean().iloc[-1] / close.iloc[-1]
    if len(close) >= 200:
        features['sma_200'] = close.rolling(window=200).mean().iloc[-1] / close.iloc[-1]
    
    # EMA
    features['ema_12'] = close.ewm(span=12).mean().iloc[-1] / close.iloc[-1]
    features['ema_26'] = close.ewm(span=26).mean().iloc[-1] / close.iloc[-1]
    
    # RSI
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    features['rsi'] = rsi.iloc[-1]
    
    # MACD
    ema_12 = close.ewm(span=12).mean()
    ema_26 = close.ewm(span=26).mean()
    macd_line = ema_12 - ema_26
    signal_line = macd_line.ewm(span=9).mean()
    features['macd'] = macd_line.iloc[-1]
    features['macd_signal'] = signal_line.iloc[-1]
    
    # Bollinger Bands
    sma = close.rolling(window=20).mean()
    std = close.rolling(window=20).std()
    upper_band = sma + (std * 2)
    lower_band = sma - (std * 2)
    current_price = close.iloc[-1]
    features['bb_position'] = (current_price - lower_band.iloc[-1]) / (upper_band.iloc[-1] - lower_band.iloc[-1])
    features['bb_width'] = (upper_band.iloc[-1] - lower_band.iloc[-1]) / close.iloc[-1]
    
    # Momentum
    if len(close) >= 10:
        features['momentum_10d'] = (close.iloc[-1] / close.iloc[-10]) - 1
    if len(close) >= 20:
        features['momentum_20d'] = (close.iloc[-1] / close.iloc[-20]) - 1
    
    # Volume indicators
    volume = group['Volume']
    features['volume_ratio'] = volume.iloc[-20:].mean() / volume.mean()
    
    # Volume trend
    volumes = volume.iloc[-20:]
    x = np.arange(len(volumes))
    if len(volumes) > 0:
        slope, _ = np.polyfit(x, volumes, 1)
        features['volume_trend'] = slope / volumes.mean()
    
    return pd.Series(features)

def calculate_statistical_features(group):
    """Calculate statistical features"""
    features = {}
    
    returns = group['Daily_Return'].dropna()
    
    if len(returns) < 30:
        return pd.Series(features)
    
    # Skewness and Kurtosis
    features['skewness'] = stats.skew(returns)
    features['kurtosis'] = stats.kurtosis(returns)
    
    # Autocorrelation
    if len(returns) > 5:
        features['autocorr_lag1'] = returns.autocorr(lag=1)
        features['autocorr_lag5'] = returns.autocorr(lag=5)
    
    # Hurst exponent
    try:
        lags = range(2, 20)
        tau = []
        lagvec = []
        
        for lag in lags:
            pp = np.array(returns[lag:])
            pp1 = np.array(returns[:-lag])
            tau.append(np.sqrt(np.std(np.subtract(pp, pp1))))
            lagvec.append(lag)
        
        if len(tau) > 0:
            poly = np.polyfit(np.log(lagvec), np.log(tau), 1)
            features['hurst_exponent'] = poly[0] * 2.0
    except:
        pass
    
    return pd.Series(features)

def calculate_liquidity_features(group):
    """Calculate liquidity metrics"""
    features = {}
    
    if len(group) < 30:
        return pd.Series(features)
    
    # Volume metrics
    features['avg_volume'] = group['Volume'].mean()
    features['avg_dollar_volume'] = (group['Close'] * group['Volume']).mean()
    features['volume_volatility'] = group['Volume'].std() / group['Volume'].mean()
    
    # Amihud illiquidity ratio
    returns = group['Daily_Return'].dropna()
    dollar_volume = group['Close'] * group['Volume']
    illiquidity = (returns.abs() / dollar_volume).replace([np.inf, -np.inf], np.nan).mean()
    features['amihud_illiquidity'] = illiquidity * 1e6
    
    # Roll measure
    try:
        cov = returns.autocorr(lag=1) * returns.var()
        if cov < 0:
            features['roll_measure'] = 2 * np.sqrt(-cov)
    except:
        pass
    
    return pd.Series(features)

def calculate_tail_risk_features(group):
    """Calculate tail risk metrics"""
    features = {}
    
    returns = group['Daily_Return'].dropna()
    
    if len(returns) < 100:
        return pd.Series(features)
    
    # Tail ratio
    right_tail = returns.quantile(0.95)
    left_tail = returns.quantile(0.05)
    if left_tail != 0:
        features['tail_ratio'] = abs(right_tail / left_tail)
    
    # Expected shortfall
    features['expected_shortfall'] = returns[returns <= returns.quantile(0.05)].mean()
    
    # Gain-to-pain ratio
    gains = returns[returns > 0].sum()
    pains = abs(returns[returns < 0].sum())
    if pains > 0:
        features['gain_to_pain'] = gains / pains
    
    # Win rate
    features['win_rate'] = (returns > 0).sum() / len(returns)
    
    # Extreme values
    features['max_daily_gain'] = returns.max()
    features['max_daily_loss'] = returns.min()
    
    return pd.Series(features)

def calculate_price_features(group):
    """Calculate price-based features"""
    features = {}
    
    # Latest price info
    features['latest_price'] = group['Close'].iloc[-1]
    features['latest_volume'] = group['Volume'].iloc[-1]
    
    # Price statistics
    features['min_price'] = group['Close'].min()
    features['max_price'] = group['Close'].max()
    features['avg_price'] = group['Close'].mean()
    features['price_range'] = features['max_price'] - features['min_price']
    features['price_range_pct'] = features['price_range'] / features['avg_price']
    
    return pd.Series(features)

features_list = []

for ticker, group in df.groupby('Ticker'):
    print(f"Processing {ticker}... ", end='')
    
    try:
        features = {'ticker': ticker}
        
        # Get categorical features
        features['sector'] = group['sector'].iloc[0] if 'sector' in group.columns else 'Unknown'
        features['industry'] = group['industry'].iloc[0] if 'industry' in group.columns else 'Unknown'
        features['company'] = group['company'].iloc[0] if 'company' in group.columns else 'Unknown'
        
        # Calculate all numeric features
        features.update(calculate_returns_features(group).to_dict())
        features.update(calculate_risk_features(group).to_dict())
        features.update(calculate_risk_adjusted_features(group).to_dict())
        features.update(calculate_technical_features(group).to_dict())
        features.update(calculate_statistical_features(group).to_dict())
        features.update(calculate_liquidity_features(group).to_dict())
        features.update(calculate_tail_risk_features(group).to_dict())
        features.update(calculate_price_features(group).to_dict())
        
        features_list.append(features)
        print(f"✓ {len(features)} features")
        
    except Exception as e:
        print(f"✗ Error: {e}")

# Create final features DataFrame
features_df = pd.DataFrame(features_list)

print(f"\n{'='*60}")
print(f"FEATURE EXTRACTION COMPLETE")
print(f"{'='*60}")
print(f"Total stocks: {len(features_df)}")
print(f"Total features: {len(features_df.columns)}")
print(f"\nFeature columns:")
print(features_df.columns.tolist())


# Save to CSV and Excel
features_df.to_excel('sp500_features.xlsx', index=False)
features_df.to_csv('sp500_features.csv', index=False)

Processing A... ✓ 61 features
Processing AAPL... ✓ 60 features
Processing ABBV... ✓ 60 features
Processing ABNB... ✓ 61 features
Processing ABT... ✓ 61 features
Processing ACGL... ✓ 60 features
Processing ACN... ✓ 61 features
Processing ADBE... ✓ 61 features
Processing ADI... ✓ 61 features
Processing ADM... ✓ 60 features
Processing ADP... ✓ 61 features
Processing ADSK... ✓ 60 features
Processing AEE... ✓ 61 features
Processing AEP... ✓ 60 features
Processing AES... ✓ 61 features
Processing AFL... ✓ 61 features
Processing AIG... ✓ 61 features
Processing AIZ... ✓ 61 features
Processing AJG... ✓ 60 features
Processing AKAM... ✓ 61 features
Processing ALB... ✓ 61 features
Processing ALGN... ✓ 61 features
Processing ALL... ✓ 61 features
Processing ALLE... ✓ 61 features
Processing AMAT... ✓ 61 features
Processing AMCR... ✓ 61 features
Processing AMD... ✓ 61 features
Processing AME... ✓ 61 features
Processing AMGN... ✓ 61 features
Processing AMP... ✓ 61 features
Processing AMT... ✓ 60 feature

In [31]:
print(f"\n{'='*60}")
print("SUMMARY STATISTICS")
print(f"{'='*60}")

# Show feature summary
print("\nFeatures by category:")
print(f"  Returns: {len([c for c in features_df.columns if 'return' in c.lower()])}")
print(f"  Risk: {len([c for c in features_df.columns if 'var' in c.lower() or 'vol' in c.lower() or 'drawdown' in c.lower()])}")
print(f"  Technical: {len([c for c in features_df.columns if any(x in c.lower() for x in ['sma', 'ema', 'rsi', 'macd', 'bb'])])}")
print(f"  Statistical: {len([c for c in features_df.columns if any(x in c.lower() for x in ['skew', 'kurt', 'autocorr', 'hurst'])])}")
print(f"  Liquidity: {len([c for c in features_df.columns if 'volume' in c.lower() or 'liquidity' in c.lower()])}")
print(f"  Tail Risk: {len([c for c in features_df.columns if any(x in c.lower() for x in ['tail', 'shortfall', 'gain_to_pain', 'win_rate'])])}")
print(f"{'\n'+'='*60+'\n'}")


SUMMARY STATISTICS

Features by category:
  Returns: 8
  Risk: 13
  Technical: 10
  Statistical: 5
  Liquidity: 7
  Tail Risk: 4




### LOAD AND EXPLORE DATA

In [37]:
stock_df = pd.read_csv('sp500_features.csv')

print(f"\nDataset Shape: {stock_df.shape}")
print(f"Number of stocks: {len(stock_df['ticker'].unique())}")
print(f"Number of features: {len(stock_df.columns)}")

# Get categorical features
categorical_cols = ['ticker', 'company', 'sector', 'industry']
stock_df_categorical = stock_df[categorical_cols].copy()

# Get numerical features
numerical_cols = [col for col in stock_df.columns if col not in categorical_cols]
stock_df_numerical = stock_df[numerical_cols].copy()

print(f"\nNumerical features: {len(numerical_cols)}")
print(f"Categorical features: {len(categorical_cols)}")

# Check for missing values
missing_vals = stock_df.isnull().sum()
missing_per = ((stock_df.isnull().sum() / len(stock_df)) * 100).round(2)

missing_df = pd.DataFrame({'Missing Count': missing_vals,'Percentage (%)': missing_per}) # Combine into a DataFrame for better visualization
missing_df[missing_df['Missing Count'] > 0].sort_values(by='Missing Count', ascending=False)


Dataset Shape: (500, 61)
Number of stocks: 500
Number of features: 61

Numerical features: 57
Categorical features: 4


,Missing Count,Percentage (%)
roll_measure,134,26.8
